# [第6章: 機械学習](https://nlp100.github.io/ja/ch06.html)
本章では，Fabio Gasparetti氏が公開している[News Aggregator Data Set](https://archive.ics.uci.edu/ml/datasets/News+Aggregator)を用い，ニュース記事の見出しを「ビジネス」「科学技術」「エンターテイメント」「健康」のカテゴリに分類するタスク（カテゴリ分類）に取り組む．

## [50. データの入手・整形Permalink](https://nlp100.github.io/ja/ch06.html#50-%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E5%85%A5%E6%89%8B%E6%95%B4%E5%BD%A2)
[News Aggregator Data Set](https://archive.ics.uci.edu/ml/datasets/News+Aggregator)をダウンロードし、以下の要領で学習データ（`train.txt`），検証データ（`valid.txt`），評価データ（`test.txt`）を作成せよ．
1. ダウンロードしたzipファイルを解凍し，`readme.txt`の説明を読む．
2. 情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．
3. 抽出された事例をランダムに並び替える．
4. 抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれ`train.txt`，`valid.txt`，`test.txt`というファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．

学習データと評価データを作成したら，各カテゴリの事例数を確認せよ．

In [1]:
!ls Input/newsCorpora.csv

Input/newsCorpora.csv


In [2]:
import pandas as pd

In [3]:
# 2.
df = pd.read_table("Input/newsCorpora.csv",
                   names=("ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"))
df = df[df["PUBLISHER"].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]
df.head(5)

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
12,13,Europe reaches crunch point on banking union,http://in.reuters.com/article/2014/03/10/eu-ba...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501755
13,14,ECB FOCUS-Stronger euro drowns out ECB's messa...,http://in.reuters.com/article/2014/03/10/ecb-p...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501948
19,20,"Euro Anxieties Wane as Bunds Top Treasuries, S...",http://www.businessweek.com/news/2014-03-10/ge...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503148
20,21,Noyer Says Strong Euro Creates Unwarranted Eco...,http://www.businessweek.com/news/2014-03-10/no...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503366
29,30,REFILE-Bad loan triggers key feature in ECB ba...,http://in.reuters.com/article/2014/03/10/euroz...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470505070


In [4]:
# 3.
df = df.sample(frac=1, random_state=50).reset_index(drop=True)
df.head(5)

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,219009,Pamela Anderson swaps pixie cut for long exten...,http://www.dailymail.co.uk/tvshowbiz/article-2...,Daily Mail,e,dqVOzSR6wswC3KMbJmSz6vfIbh0sM,www.dailymail.co.uk,1400459604154
1,71193,UPDATE 2-Euro zone inflation drops to lowest s...,http://www.reuters.com/article/2014/03/31/euro...,Reuters,b,d4X9eAjxdXsfdZMCYElnANaPvMRNM,www.reuters.com,1396286287015
2,276648,RPT-Fitch affirms Aneka Gas Industri at 'A-(id...,http://in.reuters.com/article/2014/06/11/fitch...,Reuters,b,dAZC0aRMk1ONyvMUjTyEB_iZmrZLM,in.reuters.com,1402654001555
3,398540,FOREX-Dollar sits tight near six-month high ah...,http://in.reuters.com/article/2014/07/29/marke...,Reuters,b,dnm3lDxHwNg5cHMjrmBt-YZVfC8ZM,in.reuters.com,1406714427512
4,351236,"Despite Stabbing At Meek Mill's BET Pre-Party,...",http://www.contactmusic.com/article/bet-awards...,Contactmusic.com,e,dH1n5nW6X6xNjwMIL0H2ry9UyfIfM,www.contactmusic.com,1404393924266


In [5]:
# 4.
row = len(df)

df_train = df[:int(row*0.8)][["CATEGORY", "TITLE"]]
df_train.to_csv("Output/Chapter06/train.txt", sep="\t", header=False, index=False)

df_valid = df[int(row*0.8):int(row*0.9)][["CATEGORY", "TITLE"]]
df_valid.to_csv("Output/Chapter06/valid.txt", sep="\t", header=False, index=False)

df_test = df[int(row*0.9):][["CATEGORY", "TITLE"]]
df_test.to_csv("Output/Chapter06/test.txt", sep="\t", header=False, index=False)

In [6]:
args = (row, len(df_train), len(df_valid), len(df_test))
"total:%d, train:%d, valid:%d, test:%d" % args

'total:13340, train:10672, valid:1334, test:1334'

## [51. 特徴量抽出](https://nlp100.github.io/ja/ch06.html#51-%E7%89%B9%E5%BE%B4%E9%87%8F%E6%8A%BD%E5%87%BA)
学習データ，検証データ，評価データから特徴量を抽出し，それぞれ`train.feature.txt`，`valid.feature.txt`，`test.feature.txt`というファイル名で保存せよ． なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．

## [52. 学習](https://nlp100.github.io/ja/ch06.html#52-%E5%AD%A6%E7%BF%92)
51で構築した学習データを用いて，ロジスティック回帰モデルを学習せよ．

## [53. 予測](https://nlp100.github.io/ja/ch06.html#53-%E4%BA%88%E6%B8%AC)
52で学習したロジスティック回帰モデルを用い，与えられた記事見出しからカテゴリとその予測確率を計算するプログラムを実装せよ．

## [54. 正解率の計測](https://nlp100.github.io/ja/ch06.html#54-%E6%AD%A3%E8%A7%A3%E7%8E%87%E3%81%AE%E8%A8%88%E6%B8%AC)
52で学習したロジスティック回帰モデルの正解率を，学習データおよび評価データ上で計測せよ．

## [55. 混同行列の作成](https://nlp100.github.io/ja/ch06.html#55-%E6%B7%B7%E5%90%8C%E8%A1%8C%E5%88%97%E3%81%AE%E4%BD%9C%E6%88%90)
52で学習したロジスティック回帰モデルの混同行列（confusion matrix）を，学習データおよび評価データ上で作成せよ．

## [56. 適合率，再現率，F1スコアの計測](https://nlp100.github.io/ja/ch06.html#56-%E9%81%A9%E5%90%88%E7%8E%87%E5%86%8D%E7%8F%BE%E7%8E%87f1%E3%82%B9%E3%82%B3%E3%82%A2%E3%81%AE%E8%A8%88%E6%B8%AC)
52で学習したロジスティック回帰モデルの適合率，再現率，F1スコアを，評価データ上で計測せよ．カテゴリごとに適合率，再現率，F1スコアを求め，カテゴリごとの性能をマイクロ平均（micro-average）とマクロ平均（macro-average）で統合せよ．

## [57. 特徴量の重みの確認](https://nlp100.github.io/ja/ch06.html#57-%E7%89%B9%E5%BE%B4%E9%87%8F%E3%81%AE%E9%87%8D%E3%81%BF%E3%81%AE%E7%A2%BA%E8%AA%8D)
52で学習したロジスティック回帰モデルの中で，重みの高い特徴量トップ10と，重みの低い特徴量トップ10を確認せよ．

## [58. 正則化パラメータの変更](https://nlp100.github.io/ja/ch06.html#58-%E6%AD%A3%E5%89%87%E5%8C%96%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%81%AE%E5%A4%89%E6%9B%B4)
ロジスティック回帰モデルを学習するとき，正則化パラメータを調整することで，学習時の過学習（overfitting）の度合いを制御できる．異なる正則化パラメータでロジスティック回帰モデルを学習し，学習データ，検証データ，および評価データ上の正解率を求めよ．実験の結果は，正則化パラメータを横軸，正解率を縦軸としたグラフにまとめよ．

## [59. ハイパーパラメータの探索](https://nlp100.github.io/ja/ch06.html#59-%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%81%AE%E6%8E%A2%E7%B4%A2)
学習アルゴリズムや学習パラメータを変えながら，カテゴリ分類モデルを学習せよ．検証データ上の正解率が最も高くなる学習アルゴリズム・パラメータを求めよ．また，その学習アルゴリズム・パラメータを用いたときの評価データ上の正解率を求めよ．